# Chapter 3: 编码注意力机制

在这个笔记本中使用的包：

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.4.0


- 本章介绍了注意力机制，这是LLM的核心引擎：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/01.webp?123" width="500px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/02.webp" width="600px">

## 3.1 长序列建模的问题

- 本节没有代码
- 由于源语言和目标语言之间的语法结构差异，逐字翻译文本是不可行的：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/03.webp" width="400px">

- 在引入变换器模型之前，编码器-解码器RNN常用于机器翻译任务。
- 在这种设置中，编码器处理源语言的一系列令牌，使用隐藏状态——一种神经网络内的中间层——来生成整个输入序列的压缩表示：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/04.webp" width="500px">

## 3.2 使用注意力机制捕获数据依赖关系

-本节没有代码
- 通过注意力机制，网络中生成文本的解码器部分能够选择性地访问所有输入令牌，这意味着在生成特定输出令牌时，某些输入令牌比其他令牌更重要：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/05.webp" width="500px">

- 变换器中的自注意力是一种旨在通过使序列中的每个位置都能够与同一序列中的每个其他位置进行交互并确定其相关性的技术，从而增强输入表示。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/06.webp" width="300px">

## 3.3 使用自注意力机制关注输入的不同部分

### 3.3.1  一个简单的无可训练权重的自注意力机制

- 本节解释了一种非常简化的自注意力机制变体，它不包含任何可训练的权重。
- 这纯粹是为了说明目的，而不是transformer中使用的注意力机制。
- 下一节，3.3.2节，将扩展这个简单的注意力机制来实现真正的自注意力机制。
- 假设我们有一个输入序列 $x^{(1)}$ 到 $x^{(T)}$
  - 输入是一段文本（例如，一句话，如“Your journey starts with one step（你的旅程从一步开始）”），已经根据第二章中描述的方式转换为标记嵌入。
  - 例如， $x^{(1)}$ 是一个表示单词 "Your" 的d维向量，依此类推。
- **目标：** 为 $x^{(1)}$ 到 $x^{(T)}$ 中的每个输入序列元素 $x^{(i)}$ 计算上下文向量 $z^{(i)}$ 其中 $z$  和 $x$ 具有相同的维度）。
    - 上下文向量 $z^{(i)}$ 是对输入 $x^{(1)}$ 到 $x^{(T)}$ 的加权和。
    - 上下文向量是针对某个特定输入的“上下文”特定的。
      - 不是将 $x^{(i)}$ 作为任意输入标记的占位符，我们将考虑第二个输入 $x^{(2)}$
      - 继续使用一个具体的例子，不是使用占位符 $z^{(i)}$, 我们考虑第二个输出上下文向量 $z^{(2)}$ 
      - 第二个上下文向量 $z^{(2)}$, 是对所有输入 $x^{(1)}$ 到 $x^{(T)}$ 的加权和，其权重相对于第二个输入元素 $x^{(2)}$ 而言
      - 注意力权重确定在计算 $z^{(2)}$ 时每个输入元素对加权和的贡献程度。
      - 简而言之，可以将 $z^{(2)}$ 视为 $x^{(2)}$ 的修改版本，它还整合了有关所有其他输入元素的信息，这些输入元素对于手头的给定任务是相关的。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/07.webp" width="400px">

- (Please note that the numbers in this figure are truncated to one
digit after the decimal point to reduce visual clutter; similarly, other figures may also contain truncated values)

- 根据惯例，未归一化（或称未标准化）的注意力权重被称为 **“注意力分数”**，而归一化（又称标准化）的注意力分数，其总和为1，被称为 **“注意力权重”**。


- 下面的代码逐步展示了上图中的过程。

<br>

- **步骤1:**  计算未归一化的注意力分数 $\omega$
- 假设我们使用第二个输入标记作为查询，即 $q^{(2)} = x^{(2)}$, ，我们通过点积计算未归一化的注意力分数
    - $\omega_{21} = x^{(1)} q^{(2)\top}$
    - $\omega_{22} = x^{(2)} q^{(2)\top}$
    - $\omega_{23} = x^{(3)} q^{(2)\top}$
    - ...
    - $\omega_{2T} = x^{(T)} q^{(2)\top}$
- 上面， $\omega$ 是希腊字母 "omega" ，用来表示未归一化的注意力分数。
    - $\omega_{21}$ 中的下标 "21" 表示输入序列元素2被用作查询，针对输入序列元素1。

- 假设我们有以下输入句子，已经按照第3章中描述的方式嵌入为3维向量（为了说明的目的，我们在这里使用了非常小的嵌入维度，以便它可以适合在页面上显示而不换行）：

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

- (In this book, we follow the common machine learning and deep learning convention where training examples are represented as rows and feature values as columns; in the case of the tensor shown above, each row represents a word, and each column represents an embedding dimension)

- 本节的主要目的是展示如何使用第二个输入序列 $x^{(2)}$ 作为查询来计算上下文向量 $z^{(2)}$

- 图中描述了这一过程的初始步骤，该步骤涉及通过点积操作计算 $x^{(2)}$ 与所有其他输入元素之间的注意力分数ω。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="400px">

- 我们以输入序列的第2个元素  $x^{(2)}$, 为例来计算上下文向量 $z^{(2)}$; ；在本节的后面，我们将推广这种方法来计算所有的上下文向量。
- 第一步是通过计算查询 $x^{(2)}$ 与所有其他输入标记的点积来计算非归一化的注意力分数：

In [3]:
query = inputs[1]  # 2nd input token is the query

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- 旁注：点积本质上是将两个向量的元素逐个相乘，然后将得到的乘积求和的简写方法：

In [4]:
res = 0.

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


- **第二步:** 将非标准化的注意力分数 ("omegas", $\omega$) 进行归一化，使它们的总和为1。
- 这里有一个简单的方法可以将非标准化的注意力分数归一化到总和为1（这是一个惯例，有助于解释，对训练稳定性也很重要）：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/09.webp" width="500px">

In [5]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


- 然而，在实际应用中，通常推荐使用softmax函数进行归一化，因为它在处理极端值时表现更好，并且在训练过程中具有更理想的梯度属性。
- 下面是一个用于缩放的softmax函数的简单实现，它也使得向量元素归一化，使得它们的总和为1：

In [6]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- 上面的简单实现可能会因为溢出和下溢问题，在输入值很大或很小时遇到数值不稳定的问题。
- 因此，在实践中，建议使用PyTorch的softmax实现，它已经针对性能进行了高度优化：

In [7]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- **步骤3**: 通过将嵌入的输入标记 $x^{(i)}$ 与注意力权重相乘，并对结果向量求和，计算上下文向量 $z^{(2)}$:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/10.webp" width="500px">

In [8]:
query = inputs[1] # 2nd input token is the query

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 计算所有输入tokens的注意力权重

#### 推广到所有输入序列标记：

- 在上面，我们计算了输入2的注意力权重和上下文向量（如下图中突出显示的行所示）。
- 接下来，我们将这个计算推广到计算所有注意力权重和上下文向量。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/11.webp" width="400px">

- (Please note that the numbers in this figure are truncated to two
digits after the decimal point to reduce visual clutter; the values in each row should add up to 1.0 or 100%; similarly, digits in other figures are truncated)

- 在自注意力中，过程从计算注意力分数开始，随后将其规范化以得出总和为1的注意力权重。
- 然后利用这些注意力权重通过输入的加权求和来生成上下文向量。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/12.webp" width="400px">

- 对所有成对元素应用前面的**步骤1**来计算未归一化的注意力分数矩阵：

In [9]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 我们可以通过矩阵乘法更有效地实现与上述相同的结果：

In [10]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 类似于之前的**步骤2**，我们对每一行进行归一化，使得每一行的值总和为1：

In [11]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- 快速验证每一行的值确实总和为1：

In [12]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)

print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- 应用前面的**步骤3**来计算所有上下文向量：

In [13]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


- 作为一个健全性检查，之前计算的上下文向量 $z^{(2)} = [0.4419, 0.6515, 0.5683]$ 可以在上面的第2行找到：

In [14]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## 3.4 使用可训练权重实现自注意力

- 一个概念框架，说明本节中开发的自注意力机制如何融入本书及本章的整体叙述和结构。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/13.webp" width="400px">

### 3.4.1 逐步计算注意力权重

- 在本节中，我们正在实现用于原始变换器架构、GPT模型以及大多数其他流行的LLMs中的自注意力机制。
- 这种自注意力机制也被称为“缩放点积注意力”。
- 总体思路与之前类似:
  - 我们想要计算上下文向量作为特定输入元素与输入向量的加权和。
  - 为了上述目的，我们需要注意力权重。
- 正如你将看到的，与早期介绍的基本注意力机制相比，只有轻微的差异：
  - 最显著的区别是引入了在模型训练期间更新的权重矩阵。
  - 这些可训练的权重矩阵至关重要，以便模型（特别是模型内部的注意力模块）可以学习产生“好”的上下文向量。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/14.webp" width="600px">

- 逐步实现自注意力机制，我们将从介绍三个训练权重矩阵 $W_q$, $W_k$, 和 $W_v$开始
- 这三个矩阵用于通过矩阵乘法将嵌入的输入令牌 $x^{(i)}$, 投影到查询向量、键向量和值向量上：

  - 查询向量： $q^{(i)} = W_q \,x^{(i)}$
  - 键向量： $k^{(i)} = W_k \,x^{(i)}$
  - 值向量： $v^{(i)} = W_v \,x^{(i)}$


- 输入 $x$ 和查询向量 $q$ 的嵌入维度可以相同也可以不同，这取决于模型的设计和具体实现。
- 在GPT模型中，输入和输出维度通常是相同的，但为了说明的目的，为了更好地理解计算，我们在这里选择了不同的输入和输出维度：

In [15]:
x_2 = inputs[1] # second input element
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2

- 下面，我们初始化了三个权重矩阵；请注意，出于说明目的，我们设置了 `requires_grad=False` 以减少输出中的混乱，但如果我们要在模型训练中使用这些权重矩阵，我们会设置 `requires_grad=True` 以在模型训练期间更新这些矩阵。

In [16]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

- 接下来我们计算查询、键和值向量：

In [17]:
query_2 = x_2 @ W_query # _2 because it's with respect to the 2nd input element
key_2 = x_2 @ W_key 
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


- 如下所示，我们成功地将6个输入标记从3D投影到2D嵌入空间：

In [18]:
keys = inputs @ W_key 
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


- 在下一步中，**步骤2**，我们通过计算查询向量和每个键向量之间的点积来计算未归一化的注意力分数：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/15.webp" width="600px">

In [19]:
keys_2 = keys[1] # Python starts index at 0
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


- 由于我们有6个输入，对于给定的查询向量，我们有6个注意力分数：

In [20]:
attn_scores_2 = query_2 @ keys.T # All attention scores for given query
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/16.webp" width="600px">

- 接下来，在**步骤3**中，我们使用我们之前使用过的 softmax 函数计算注意力权重（归一化的注意力分数，总和为1）。
- 与之前的不同之处在于，我们现在通过将注意力分数除以嵌入维度的平方根 $\sqrt{d_k}$ (例如, `d_k**0.5`)来对其进行缩放：

In [21]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/17.webp" width="600px">

- 在**步骤4**中，我们现在计算输入查询向量2的上下文向量：

In [22]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


### 3.4.2 实现一个简洁的SelfAttention类

- 综合考虑以上所有内容，我们可以实现自注意力机制如下：

In [23]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        
        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="400px">

- 我们可以使用 PyTorch 的线性层来简化上面的实现，这些层相当于矩阵乘法，如果我们禁用偏置单元。
- 使用  `nn.Linear` 而不是我们手动的  `nn.Parameter(torch.rand(...)` 方法的另一个重大优势是，`nn.Linear` 具有首选的权重初始化方案，这导致模型训练更加稳定。

In [ ]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


- 请注意， `SelfAttention_v1` 和 `SelfAttention_v2` 给出不同的输出，因为它们对权重矩阵使用不同的初始权重。

## 3.5 使用因果注意力隐藏未来词语

- 在因果注意力中，对角线以上的注意力权重被掩盖，确保对于任何给定的输入，LLM在用注意力权重计算上下文向量时无法利用未来的令牌。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/19.webp" width="400px">

### 3.5.1 应用一个因果注意力掩码

- 在本节中，我们将先前的自注意力机制转换为因果自注意力机制。
- 因果自注意力确保模型对序列中某一位置的预测只依赖于先前位置的已知输出，而不依赖于未来位置。
- 简单来说，这确保了每个后续单词的预测只应依赖于前面的单词。
- 为了实现这一点，对于每个给定的令牌（此前也称为标记），我们屏蔽掉未来的令牌（输入文本中当前令牌之后的令牌）：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/20.webp" width="600px">

- 为了说明和实现因果自注意力，让我们使用上一节中的注意力分数和权重：

In [25]:
# Reuse the query and key weight matrices of the
# SelfAttention_v2 object from the previous section for convenience
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs) 
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


- 屏蔽未来的注意力权重的最简单方法是通过使用 PyTorch 的 tril 函数创建一个掩码，使主对角线以下的元素（包括对角线本身）设置为1，而主对角线以上的元素设置为0：

In [26]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- 然后，我们可以将注意力权重与这个掩码相乘，以将对角线以上的注意力分数归零：

In [27]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


- 然而，如果像上面那样在softmax之后应用掩码，它会破坏由softmax创建的概率分布。
- Softmax确保所有输出值的总和为1。
- 在softmax之后进行掩码处理将需要重新规范化输出以使总和再次为1，这会使过程复杂化，并可能导致意外的效果。

- 为了确保行总和为1，我们可以按如下方式对注意力权重进行归一化：

In [28]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


- 尽管我们在技术上已经完成了编写因果注意力机制的工作，但让我们简要地看一下实现相同效果的更有效方法。
- 因此，我们可以在进入softmax函数之前用负无穷大掩盖对角线以上的未归一化的注意力分数，而不是将注意力权重归零并重新归一化结果：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/21.webp" width="450px">

In [29]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


- 正如我们下面所看到的，现在每一行中的注意力权重正确地再次总和为1：

In [30]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 3.5.2 使用dropout屏蔽额外的注意力权重

- 此外，我们还应用dropout来减少训练过程中的过拟合。
- Dropout可以应用在几个地方：
  - 例如，在计算注意力权重之后；
  - 或者在将注意力权重与值向量相乘之后。
- 在这里，我们将在计算注意力权重之后应用dropout掩码，因为这更常见。

- 此外，在这个特定示例中，我们使用50%的dropout率，这意味着随机屏蔽掉一半的注意力权重。（当我们稍后训练GPT模型时，我们将使用更低的dropout率，例如0.1或0.2。）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/22.webp" width="400px">

- 如果我们应用一个dropout率为0.5（50％），那么未被删除的值将按照1/0.5 = 2的因子进行相应的缩放。
- 计算公式为 1 / (1 - `dropout_rate`)

In [31]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # dropout rate of 50%
example = torch.ones(6, 6) # create a matrix of ones

print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [32]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)


- Note that the resulting dropout outputs may look different depending on your operating system; you can read more about this inconsistency [here on the PyTorch issue tracker](https://github.com/pytorch/pytorch/issues/121595)

### 3.5.3  实现一个简洁的因果自注意力类

- 现在，我们准备好实现一个可工作的自注意力实现，包括因果和dropout掩码。
- 还有一件事是实现处理包含多个输入的批次的代码，这样我们的 `CausalAttention` 类就支持我们在第2章实现的数据加载器产生的批次输出。
- 为了简单起见，为了模拟这样的批量输入，我们复制了输入文本示例：

In [33]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) # 2 inputs with 6 tokens each, and each token has embedding dimension 3

torch.Size([2, 6, 3])


In [34]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 注意，dropout仅在训练期间应用，而不是在推理期间应用。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/23.webp" width="500px">

## 3.6 将单头注意力扩展到多头注意力

### 3.6.1 堆叠多个单头注意力层

- 以下是之前实现的自注意力机制的总结（为简单起见，未显示因果和dropout掩码）。

- 这也被称为单头注意力：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/24.webp" width="400px">

- 我们简单地堆叠多个单头注意力模块以获得多头注意力模块：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/25.webp" width="400px">

- 多头注意力背后的主要思想是用不同的学习线性投影（并行）多次运行注意力机制。这允许模型共同关注来自不同表示子空间在不同位置的信息。

In [35]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- 在上面的实现中，嵌入维度为4，因为我们将`d_out=2`作为键、查询和值向量以及上下文向量的嵌入维度。而且由于我们有2个注意力头,我们有输出嵌入维度2*2=4。

### 3.6.2 实现带有权重分割的多头注意力

- 虽然上面是一个直观且功能完整的多头注意力实现（包装了早期的单头注意力`CausalAttention`实现），但我们可以编写一个名为`MultiHeadAttention`的独立类来实现相同的功能
- 对于这个独立的`MultiHeadAttention`类,我们不会连接单个注意力头
- 相反,我们创建单个W_query、W_key和W_value权重矩阵,然后将其拆分为每个注意力头的单个矩阵:

In [36]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 注意,上面实质上是一个重写的`MultiHeadAttentionWrapper`版本,更加高效。
- 由于随机权重初始化不同,因此得到的输出看起来有一点不同,但两者都是可以在我们即将实现的GPT类中使用的功能完整的实现。
- 另请注意,我们还在上面的`MultiHeadAttention`类中添加了一个线性投影层(`self.out_proj`)。这只是一个不改变维度的线性变换。在LLM实现中使用这种投影层是标准惯例,但并非是绝对必要的(最新研究表明,它可以被移除而不会影响建模性能;请参阅本章末尾的进一步阅读部分)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="400px">

- 请注意，如果您对上述内容的紧凑和高效实现感兴趣，您也可以考虑使用 PyTorch 中的 [`torch.nn.MultiheadAttention`](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html) 类

- 鉴于上述实现乍看之下可能有些复杂，让我们来看看执行 `attn_scores = queries @ keys.transpose(2, 3)`时发生了什么：

In [37]:
# (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

print(a @ a.transpose(2, 3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


- 在这种情况下,PyTorch中的矩阵乘法实现将处理4维输入张量,以便在最后2个维度(num_tokens,head_dim)之间进行矩阵乘法,然后对每个单独的头重复此操作。
- 例如,上面成为了一种更加紧凑的方式,用于单独为每个头计算矩阵乘法:

In [38]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


# 总结与要点

- 请参考 [./multihead-attention.ipynb](./multihead-attention.ipynb) ，它是数据加载器（第2章）的简洁版本，加上我们在本章实现的多头注意力类，并且在接下来的章节中训练 GPT 模型时将需要它。
- You can find the exercise solutions in [./exercise-solutions.ipynb](./exercise-solutions.ipynb)